# grid
> Set of basic operations on grids

In [ ]:
# | default_exp grid

In [ ]:
# | hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
# | export
import pandas as pd
from openlocationcode import openlocationcode as olc
from typing import Tuple

In [ ]:
#| export
def get_offset_gridchar(
    start: int, # The starting character
    offset: int # The number of steps to take from the starting character
    )-> Tuple[str, int]: # The new character and the number of wraparounds
    gridseq = [
        "2",
        "3",
        "4",
        "5",
        "6",
        "7",
        "8",
        "9",
        "C",
        "F",
        "G",
        "H",
        "J",
        "M",
        "P",
        "Q",
        "R",
        "V",
        "W",
        "X",
    ]
    if start not in gridseq:
        return None
    else:
        start_idx = gridseq.index(start)
        wraparounds = int((offset / abs(offset)) * (abs(offset) // len(gridseq)))
        local_offset = int((offset / abs(offset)) * (abs(offset) % len(gridseq)))

        # print(start_idx, wraparounds, local_offset)

        if start_idx + local_offset < 0:
            return gridseq[start_idx + local_offset + len(gridseq)], wraparounds - 1

        elif start_idx + local_offset > len(gridseq) - 1:
            return gridseq[start_idx + local_offset - len(gridseq)], wraparounds + 1

        else:
            return gridseq[start_idx + local_offset], wraparounds

In [ ]:
#| hide
# Testing various offsets
test_eq(get_offset_gridchar("C", 1), ("F", 0))
test_eq(get_offset_gridchar("C", -1), ("9", 0))
test_eq(get_offset_gridchar("C", 16), ("6", 1))
test_eq(get_offset_gridchar("C", -36), ("J", -2))

In [ ]:
#| export
def get_olc_with_offsets(
    olc_code:str,       # Open location code grid id
    v_offset:int=0,     # Vertical offset to be applied to the grid id
    h_offset:int=0      # Horizontal offset to be applied to the grid id
    )-> str:            # The new grid id after applying offsets
    if olc_code is None:
        return None
    else:
        # Split the OLC code into its component parts.
        olc_code = ''.join(olc_code.split("+"))
        olc_components = [olc_code[i:i+2] for i in range(0, len(olc_code), 2)]
        # print(olc_components)
        
        component_pointer = len(olc_components)
        wraparound = True
        
        while wraparound:
            component_pointer -= 1
            component = olc_components[component_pointer]
            v_char = component[0]
            h_char = component[1]
            
            if v_offset != 0:
                v_char, v_offset = get_offset_gridchar(component[0], v_offset)
            if h_offset != 0:
                h_char, h_offset = get_offset_gridchar(component[1], h_offset)
            
            new_component = v_char + h_char
            olc_components[component_pointer] = new_component
            wraparound = v_offset != 0 or h_offset != 0
        
        if len(olc_components) > 4:
            olc_code = ''.join(olc_components[:4]) + '+' + ''.join(olc_components[4:])
        else:
            olc_code = ''.join(olc_components)
            
        return olc_code

In [ ]:
#| hide
# Testing grids at various resolutions
test_eq(get_olc_with_offsets('7JCMHQ', -1, -1), '7JCMGP')
test_eq(get_olc_with_offsets('7JCMHQ9C+2W', 0, 0), '7JCMHQ9C+2W')
test_eq(get_olc_with_offsets('84VVHQMX', -1, -1), '84VVHQJW')
test_eq(get_olc_with_offsets('7JCMHQ9C+2W', 1, 1), '7JCMHQ9C+3X')
test_eq(get_olc_with_offsets('7JCMHQ9C+2W', -1, -1), '7JCMHQ8C+XV')

In [ ]:
# | hide
import nbdev; nbdev.nbdev_export()